### 四种处理文档的预制链实现文档对话
- stuff documents 将文档直接插入prompt中，适合文档较小或者少量文档的应用
- refine documents chain 通过循环输入文档并迭代更新答案来构建响应，一次只传递给llm一个文档，适合llm上下文不能容纳的小文档
- map reduce documents chain 先将llm链单独用于每个文档(map)，将链输入视为新文档，然后将新文档传递到单独的组合文档链以获得单个输出(reduce)，他可以选择首先压缩或折叠映射文档，以确保他们适合组合文档链（这通常需要传递给llm）。如有必要，该压缩步骤会递归执行
- map re-rank documents chain 对每个文档运行初始提示，这不仅会尝试完成任务，还会对其答案的确定性给出分数，返回得分最高的响应

In [2]:
# stuff documents
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import Tongyi
from langchain.document_loaders import PyPDFLoader
from langchain.chains import LLMChain

loader = PyPDFLoader("../ChatDoc/resource/test.pdf")

# documents = loader.load()
# documents

prompt_template = """对以下的文字进行简单的总结：{text},简单的总结："""
prompt = PromptTemplate.from_template(prompt_template)

llm = Tongyi(model='qwen-plus', temperature=0)

llm_chain = LLMChain(
    llm=llm,
    prompt=prompt
)

stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="text",
)

docs = loader.load()
result = stuff_chain.invoke({"input_documents": docs})

print(result["output_text"])


C:\Users\Administrator\AppData\Local\Temp\ipykernel_6324\3444016675.py:18: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(
C:\Users\Administrator\AppData\Local\Temp\ipykernel_6324\3444016675.py:23: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain/
  stuff_chain = StuffDocumentsChain(


该文档列出了多个城市中体检机构的详细地址及最近的地铁或公交出行建议。以下是简单的总结：

1. **广州**：包括天河、越秀、海珠、番禺、增城和花都等多个区域的体检机构地址及交通指引。
2. **武汉**：涵盖江汉、武昌、洪山、江岸、硚口、汉阳和新洲等区域的体检机构信息。
3. **深圳**：涉及南山区、福田区、罗湖区、宝安区、龙岗区和盐田区等地的体检机构位置与交通方式。
4. **北京**：列出朝阳、海淀、西城、崇文、丰台和顺义等区域的体检机构具体地址和出行建议。
5. **佛山**：禅城、南海和顺德区的体检机构信息及其交通路线。
6. **珠海**：香洲区的体检机构地址和交通指引。
7. **海南（海口）**：美兰、琼山等区域的体检机构信息。
8. **上海**：黄浦、长宁、普陀、静安和杨浦等区域的体检机构地址及交通方式。

总体上，文档提供了详细的地理位置和便捷的公共交通方案，方便用户前往各个体检机构。


In [3]:
# 使用预封装好的loader_summarize_chain 等同于上面的代码
from langchain.chains.summarize import load_summarize_chain

loader = PyPDFLoader("../ChatDoc/resource/test.pdf")
docs= loader.load()
llm = Tongyi(model='qwen-plus', temperature=0)
chain= load_summarize_chain(
    llm=llm,
    chain_type="stuff",
    verbose=True
)
chain.run(docs)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_6324\2912566480.py:12: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(docs)




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"机构 地区 行政区 地址 最近地铁/公交出行建议
广州 天河 海珠区暄悦东街23号保利中悦广场南塔3
楼 地铁4号线或者8号线万胜围地铁站D出口走800M
广州 越秀 环市东路496号广发花园大厦2、3、4楼
（可按楼层分开约） 地铁5号线动物园站C出口走600M
广州 天河 天河区珠江新城花城大道7号南天国际商
务中心3、5层（可按楼层分开约） 地铁5号线珠江新城A2出口走570M
广州 天河 天河区东方一路华港花园20-24号3层 公交B9东方一路站走80M
广州 天河 天河区林和西路161号中泰国际广场3楼
、5楼（可按楼层分开约） 地铁1号线广州东站B1出口
武汉 江汉区 江汉区后襄河北路59号1幢3层 地铁2号线汉口火车站E出口走670M
武汉 武昌区 武昌区武珞路568号南方帝园1-3F 地铁2号线宝通寺站 A出口走540M
武汉 洪山区 洪山区民族大道219号同学广场A区2层 公交758路   (或 759路 )民族大道中南民族大学站走
90M
武汉 江汉区 江汉区后襄河北路59号1幢3层 地铁2号线汉口火车站B出口走820M
深圳 南山区 深圳市南山区高新科技园中区科苑路科兴
科学院B座3单元3A层 下楼即到
深圳 福田区 深圳福田区滨河路北彩田路东交汇处联合
广场B座裙楼B201/203 地铁1号线岗厦站站E出口走740M
深圳 罗湖区 深圳市罗湖区宝安南路3044号天地大厦1
、3层 地铁1号线大剧院D出口走1.5KM
深圳 宝安区 深圳市宝安区西乡街道永丰社区新湖路
4008号蘅芳科技大厦3楼 地铁3号线红岭站C2口出走1KM
深圳 南山区 深圳市福田区深南中路3039号深圳国际
文化大厦3A层 地铁1号线华强路站 C口出900M
北京 朝阳区 朝阳区慧忠北里105楼B段京师科技大厦
第三层
公交579路   (或 645路 510路 )北京师范大学南门站
下车走270米
北京 海淀区 海淀区羊坊店路21号3层 地铁7号线北京西站 北口出走500M

"The document provides detailed addresses and transportation suggestions for various health check-up institutions under two major brands, **Ai Kang Guo Pin** and **Mei Nian Da Jian Kang**, across multiple cities in China. The locations span several provinces and municipalities, including Guangzhou, Wuhan, Shenzhen, Beijing, Shanghai, Foshan, Haikou, and Zhuhai. Each entry includes the institution's name, region, district, specific address, and the nearest subway or bus station with walking distance instructions. \n\nFor example:\n- In **Guangzhou**, Ai Kang Guo Pin has branches in districts like Tianhe, Haizhu, and Yuexiu, with directions from nearby metro stations.\n- In **Wuhan**, both Ai Kang Guo Pin and Mei Nian Da Jian Kang have multiple branches, accessible via metro lines or buses.\n- In **Shenzhen**, Mei Nian Da Jian Kang offers services in areas such as Nanshan, Futian, Luohu, and Bao'an, with public transport options provided.\n- In **Beijing**, locations are spread across di

In [4]:
# reifne 通过循环引用llm，将文档不断投喂，并产生各种中间答案，适合逻辑有上下文关联的文档，不适合交叉引用的文档

from langchain.prompts import PromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain_community.llms import Tongyi
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain

# load
loader = PyPDFLoader("../ChatDoc/resource/test.pdf")
docs = loader.load()

# split
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000,
    chunk_overlap=200,
)
split_docs = text_splitter.split_documents(docs)

# prompt
prompt_template = """对以下的文字进行简单的总结：{text},简单的总结："""
prompt = PromptTemplate.from_template(prompt_template)

refine_template = ("你的任务是产生最终摘要，\n我们已经提供了一个到某个特定特定点的现有摘要:{existing_answer}\n我们有机会通过下面的一些更多上下文来完善现有的回答（仅在需要时使用）：{text}，根据新的上下文，用中文完善原始回答，如果上下文没有用处，返回原始回答")
refine_prompt = PromptTemplate.from_template(refine_template)

llm = Tongyi(model='qwen-plus', temperature=0)

chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    return_intermediate_steps=True,
    verbose=True,
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    input_key="documents",
    output_key="output_text",
)

In [ ]:
result = chain({"documents": split_docs}, return_only_outputs=True)
print(result["output_text"])

C:\Users\Administrator\AppData\Local\Temp\ipykernel_6324\875153788.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain({"documents": split_docs}, return_only_outputs=True)




> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
对以下的文字进行简单的总结：机构 地区 行政区 地址 最近地铁/公交出行建议
广州 天河 海珠区暄悦东街23号保利中悦广场南塔3
楼 地铁4号线或者8号线万胜围地铁站D出口走800M
广州 越秀 环市东路496号广发花园大厦2、3、4楼
（可按楼层分开约） 地铁5号线动物园站C出口走600M
广州 天河 天河区珠江新城花城大道7号南天国际商
务中心3、5层（可按楼层分开约） 地铁5号线珠江新城A2出口走570M
广州 天河 天河区东方一路华港花园20-24号3层 公交B9东方一路站走80M
广州 天河 天河区林和西路161号中泰国际广场3楼
、5楼（可按楼层分开约） 地铁1号线广州东站B1出口
武汉 江汉区 江汉区后襄河北路59号1幢3层 地铁2号线汉口火车站E出口走670M
武汉 武昌区 武昌区武珞路568号南方帝园1-3F 地铁2号线宝通寺站 A出口走540M
武汉 洪山区 洪山区民族大道219号同学广场A区2层 公交758路   (或 759路 )民族大道中南民族大学站走
90M
武汉 江汉区 江汉区后襄河北路59号1幢3层 地铁2号线汉口火车站B出口走820M
深圳 南山区 深圳市南山区高新科技园中区科苑路科兴
科学院B座3单元3A层 下楼即到
深圳 福田区 深圳福田区滨河路北彩田路东交汇处联合
广场B座裙楼B201/203 地铁1号线岗厦站站E出口走740M
深圳 罗湖区 深圳市罗湖区宝安南路3044号天地大厦1
、3层 地铁1号线大剧院D出口走1.5KM
深圳 宝安区 深圳市宝安区西乡街道永丰社区新湖路
4008号蘅芳科技大厦3楼 地铁3号线红岭站C2口出走1KM
深圳 南山区 深圳市福田区深南中路3039号深圳国际
文化大厦3A层 地铁1号线华强路站 C口出900M
北京 朝阳区 朝阳区慧忠北里105楼B段京师科技大厦
第三层
公交579路   (或 645路 510路 )北京师范大学南门站
下车走270米
北京 海淀区 海淀区羊坊店路21号3层 地铁7号线北京西站 北口出走500M
北京 朝阳区 朝阳区建华南路17号现代柏联大厦二层 公交9

In [ ]:
# map reduce
from langchain.chains.combine_documents.map_reduce import MapReduceDocumentsChain
from langchain.chains.combine_documents.reduce import ReduceDocumentsChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import Tongyi
from langchain.chains.llm import LLMChain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

# 1. 加载文档
loader = PyPDFLoader("../ChatDoc/resource/test.pdf")
docs = loader.load()

# 2. 分割文档 - 将大文档拆分为适合处理的小块
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000,  # 每个块的大小
    chunk_overlap=0,  # 块之间不重叠
)
split_docs = text_splitter.split_documents(docs)

# 3. Map阶段 - 对每个文档块单独处理
map_template = """对以下的文字进行简单的总结：{text},简单的总结："""
map_prompt = PromptTemplate.from_template(map_template)
llm = Tongyi(model='qwen-plus', temperature=0)

# Map链 - 处理单个文档块
map_chain = LLMChain(
    llm=llm,
    prompt=map_prompt
)

# 4. Reduce阶段 - 合并所有Map结果
reduce_template = """以下是一个摘要集合：{doc_summaries},将上述摘要与所有关键细节进行总结，总结："""
reduce_prompt = PromptTemplate.from_template(reduce_template)

# Reduce链 - 处理合并后的摘要
reduce_chain = LLMChain(
    llm=llm,
    prompt=reduce_prompt
)

# 5. Stuff链 - 准备最终合并
# 注意：这里使用reduce_chain作为LLM链
stuff_chain = StuffDocumentsChain(
    llm_chain=reduce_chain,
    document_variable_name="doc_summaries",
)

# 6. Reduce文档链 - 处理可能的大文档情况
reduce_final_chain = ReduceDocumentsChain(
    combine_documents_chain=stuff_chain,  # 最终合并链
    collapse_documents_chain=stuff_chain,  # 折叠文档链（处理超长文档）
    token_max=4000  # 最大token限制
)

# 7. MapReduce主链 - 整合Map和Reduce
map_reduce_chain = MapReduceDocumentsChain(
    llm_chain=map_chain,  # Map阶段链
    reduce_documents_chain=reduce_final_chain,  # Reduce阶段链
    document_variable_name="text",  # 文档输入变量名
    return_intermediate_steps=False  # 不返回中间步骤
)

# 8. 执行链
# 注意：输入必须是字典格式，键为"input_documents"
result = map_reduce_chain.invoke({"input_documents": split_docs})

# 9. 输出结果
print(result["output_text"])

In [6]:
# map re-rank documents chain

from langchain.chains.combine_documents.map_rerank import MapRerankDocumentsChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import Tongyi
from langchain.document_loaders import PyPDFLoader
from langchain.chains import LLMChain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

llm = Tongyi(model='qwen-plus', temperature=0)

loader = PyPDFLoader("../ChatDoc/resource/test.pdf")
docs = loader.load()

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500,
    chunk_overlap=0,
)
split_docs = text_splitter.split_documents(docs)

chain=load_qa_with_sources_chain(
    llm=llm,
    chain_type="map_rerank",
    metadata_keys=["source"],
    return_intermediate_steps=True,
)
print(chain)

verbose=False llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, output_parser=RegexParser(regex='(.*?)\\nScore: (\\d*)', output_keys=['answer', 'score']), partial_variables={}, template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nIn addition to giving an answer, also return a score of how fully it answered the user's question. This should be in the following format:\n\nQuestion: [question here]\nHelpful Answer: [answer here]\nScore: [score between 0 and 100]\n\nHow to determine the score:\n- Higher is a better answer\n- Better responds fully to the asked question, with sufficient level of detail\n- If you do not know the answer based on the context, that should be a score of 0\n- Don't be overconfident!\n\nExample #1\n\nContext:\n---------\nApples are red\n---------\nQuestion: what color are apples?\nHe

In [7]:
query = "what is this document about? Answer in Chinese."
result = chain({"input_documents": split_docs, "question": query})

C:\Users\Administrator\AppData\Local\Temp\ipykernel_11624\3391996286.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain({"input_documents": split_docs, "question": query})
d:\My Documents\Desktop\langchain\.venv\lib\site-packages\langchain\chains\llm.py:370: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [8]:
result

{'input_documents': [Document(metadata={'producer': 'Microsoft® Excel® 2016', 'creator': 'Microsoft® Excel® 2016', 'creationdate': '2024-11-07T15:53:15+08:00', 'author': '伍润洪', 'moddate': '2024-11-07T15:53:15+08:00', 'source': '../ChatDoc/resource/test.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='机构 地区 行政区 地址 最近地铁/公交出行建议\n广州 天河 海珠区暄悦东街23号保利中悦广场南塔3\n楼 地铁4号线或者8号线万胜围地铁站D出口走800M\n广州 越秀 环市东路496号广发花园大厦2、3、4楼\n（可按楼层分开约） 地铁5号线动物园站C出口走600M\n广州 天河 天河区珠江新城花城大道7号南天国际商\n务中心3、5层（可按楼层分开约） 地铁5号线珠江新城A2出口走570M\n广州 天河 天河区东方一路华港花园20-24号3层 公交B9东方一路站走80M\n广州 天河 天河区林和西路161号中泰国际广场3楼\n、5楼（可按楼层分开约） 地铁1号线广州东站B1出口\n武汉 江汉区 江汉区后襄河北路59号1幢3层 地铁2号线汉口火车站E出口走670M\n武汉 武昌区 武昌区武珞路568号南方帝园1-3F 地铁2号线宝通寺站 A出口走540M\n武汉 洪山区 洪山区民族大道219号同学广场A区2层 公交758路   (或 759路 )民族大道中南民族大学站走\n90M\n武汉 江汉区 江汉区后襄河北路59号1幢3层 地铁2号线汉口火车站B出口走820M\n深圳 南山区 深圳市南山区高新科技园中区科苑路科兴\n科学院B座3单元3A层 下楼即到\n深圳 福田区 深圳福田区滨河路北彩田路东交汇处联合\n广场B座裙楼B201/203 地铁1号线岗厦站站E出口走740M\n深圳 罗湖区 深圳市罗湖区宝安南路3044号天地大厦1\n、3层 地铁1号线大剧院D出口走1.5KM\n深圳 宝安区 深圳市宝安区西乡街道永丰社区新湖路\n4